In [78]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.optimizers import Adam

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
cd_score = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/5score 기반 모델 생성 /csv/click_del_score.csv', encoding='cp949')
cd_score

,subsr,series_nm,vod_id,ct_cl,genre_of_ct_cl,category,cd_score
0,65968000,핑크퐁 동물 동요,882,키즈,기타,키즈,1.000000
1,63086000,지니강이 플러스 시즌7,1622,키즈,기타,키즈,0.131098
2,63086000,지니강이 플러스 시즌9,2043,키즈,기타,키즈,0.149495
3,62733000,엘리가간다 시즌12,1958,키즈,기타,키즈,0.500000
4,63086000,지니강이 플러스 시즌8,1596,키즈,기타,키즈,0.152544
...,...,...,...,...,...,...,...
7352,64982000,포켓몬스터 XY and Z,2918,키즈,애니메이션,키즈,0.014493
7353,66864000,포켓몬스터 썬 앤 문 3,3135,키즈,애니메이션,키즈,0.045455
7354,59917000,라바 인 뉴욕,10,키즈,오락,키즈,0.500000
7355,63086000,사이틴 시즌1,3468,키즈,학습,키즈,0.006098


In [81]:
# 사용자 ID와 VOD ID를 숫자 인덱스로 매핑
score = cd_score.copy()
user_ids = cd_score['subsr'].unique().tolist()
vod_ids = cd_score['vod_id'].unique().tolist()
cd_score['user'] = cd_score['subsr'].map({id: idx for idx, id in enumerate(user_ids)})
cd_score['vod'] = cd_score['vod_id'].map({id: idx for idx, id in enumerate(vod_ids)})
cd_score

,subsr,series_nm,vod_id,ct_cl,genre_of_ct_cl,category,cd_score,user,vod
0,65968000,핑크퐁 동물 동요,882,키즈,기타,키즈,1.000000,0,0
1,63086000,지니강이 플러스 시즌7,1622,키즈,기타,키즈,0.131098,1,1
2,63086000,지니강이 플러스 시즌9,2043,키즈,기타,키즈,0.149495,1,2
3,62733000,엘리가간다 시즌12,1958,키즈,기타,키즈,0.500000,2,3
4,63086000,지니강이 플러스 시즌8,1596,키즈,기타,키즈,0.152544,1,4
...,...,...,...,...,...,...,...,...,...
7352,64982000,포켓몬스터 XY and Z,2918,키즈,애니메이션,키즈,0.014493,236,2675
7353,66864000,포켓몬스터 썬 앤 문 3,3135,키즈,애니메이션,키즈,0.045455,651,2676
7354,59917000,라바 인 뉴욕,10,키즈,오락,키즈,0.500000,898,2677
7355,63086000,사이틴 시즌1,3468,키즈,학습,키즈,0.006098,1,2678


In [82]:
cd_score['subsr'].value_counts()

60939000    446
66705000    230
64984000    189
64124000    162
64659000    152
           ... 
63748000      1
66111000      1
62017000      1
65358000      1
65968000      1
Name: subsr, Length: 899, dtype: int64

In [83]:
# 사용자별 등장 횟수 계산
user_counts = cd_score['subsr'].value_counts()

# 5개 미만으로 등장하는 사용자 필터링
users_up5 = user_counts[user_counts >= 5].index
user_down5 = user_counts[user_counts < 5].index

# 5개 미만으로 등장하는 사용자를 포함하지 않는 행 선택
score_filtered = cd_score[cd_score['subsr'].isin(users_up5)]
score_filtered

,subsr,series_nm,vod_id,ct_cl,genre_of_ct_cl,category,cd_score,user,vod
1,63086000,지니강이 플러스 시즌7,1622,키즈,기타,키즈,0.131098,1,1
2,63086000,지니강이 플러스 시즌9,2043,키즈,기타,키즈,0.149495,1,2
4,63086000,지니강이 플러스 시즌8,1596,키즈,기타,키즈,0.152544,1,4
5,63086000,헤이지니 럭키강이 Part1,4036,키즈,기타,키즈,0.006098,1,5
6,63086000,우당탕탕 에디의 실험실,1834,키즈,기타,키즈,0.370681,1,6
...,...,...,...,...,...,...,...,...,...
7351,60040000,포켓몬스터: 리코와 로드의 모험,1273,키즈,애니메이션,키즈,0.076923,48,2657
7352,64982000,포켓몬스터 XY and Z,2918,키즈,애니메이션,키즈,0.014493,236,2675
7353,66864000,포켓몬스터 썬 앤 문 3,3135,키즈,애니메이션,키즈,0.045455,651,2676
7355,63086000,사이틴 시즌1,3468,키즈,학습,키즈,0.006098,1,2678


In [84]:
user_down5

Int64Index([66431000, 66110000, 65941000, 64252000, 61442000, 60496000,
            66503000, 62366000, 65579000, 64756000,
            ...
            65626000, 66182000, 61467000, 66573000, 66265000, 63748000,
            66111000, 62017000, 65358000, 65968000],
           dtype='int64', length=609)

In [85]:
down_filtered = cd_score[cd_score['subsr'].isin(user_down5)]
down_filtered['subsr'].value_counts()

65121000    4
65043000    4
64401000    4
60496000    4
66431000    4
           ..
66632000    1
64040000    1
66163000    1
64070000    1
65968000    1
Name: subsr, Length: 609, dtype: int64

In [86]:
score_filtered['subsr'].value_counts()

60939000    446
66705000    230
64984000    189
64124000    162
64659000    152
           ... 
65967000      5
59933000      5
64300000      5
66402000      5
62208000      5
Name: subsr, Length: 290, dtype: int64

In [87]:
# train, test 분할
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for _, group in score_filtered.groupby('subsr'):
    interactions = group.iloc[1:3]
    train_data = train_data.append(group.drop(interactions.index))
    test_data = test_data.append(interactions)

# 훈련 데이터와 테스트 데이터 확인
print(f"Train data shape: {train_data}")
print(f"Test data shape: {test_data}")

<ipython-input-87-e995a9d9bd70>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(group.drop(interactions.index))
<ipython-input-87-e995a9d9bd70>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(interactions)
<ipython-input-87-e995a9d9bd70>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(group.drop(interactions.index))
<ipython-input-87-e995a9d9bd70>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(interactions)
<ipython-input-87-e995a9d9bd70>:7: FutureWarning: The frame.append method is deprecated and will be removed 

Train data shape:          subsr           series_nm  vod_id  ct_cl genre_of_ct_cl category  \
2832  59879000           소방서 옆 경찰서    2092  TV드라마             기타   TV프로그램   
2986  59879000       슬기로운 의사생활 시즌2    2891  TV드라마             기타   TV프로그램   
2991  59879000             나의 해방일지    1859  TV드라마             기타   TV프로그램   
3022  59879000                나쁜엄마    3403  TV드라마             기타   TV프로그램   
3168  59879000                쌍갑포차    3115  TV드라마             기타   TV프로그램   
...        ...                 ...     ...    ...            ...      ...   
5902  67160000               비공식작전    2057     영화            드라마       영화   
6327  67160000  스파이더맨: 어크로스 더 유니버스    2147     영화          애니메이션       영화   
6643  67160000              메가로돈 2    4018     영화        액션/어드벤쳐       영화   
6933  67160000                 귀공자    2058     영화        액션/어드벤쳐       영화   
7186  67160000                  바비    3523     영화            코미디       영화   

      cd_score  user   vod  
2832  0.288905   546  1066  

<ipython-input-87-e995a9d9bd70>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(interactions)
<ipython-input-87-e995a9d9bd70>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(group.drop(interactions.index))
<ipython-input-87-e995a9d9bd70>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(interactions)
<ipython-input-87-e995a9d9bd70>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(group.drop(interactions.index))
<ipython-input-87-e995a9d9bd70>:8: FutureWarning: The frame.append method is deprecated and will be removed 

In [88]:
score_filtered.head(3)

,subsr,series_nm,vod_id,ct_cl,genre_of_ct_cl,category,cd_score,user,vod
1,63086000,지니강이 플러스 시즌7,1622,키즈,기타,키즈,0.131098,1,1
2,63086000,지니강이 플러스 시즌9,2043,키즈,기타,키즈,0.149495,1,2
4,63086000,지니강이 플러스 시즌8,1596,키즈,기타,키즈,0.152544,1,4


In [89]:
# NCF 모델 구성
user_input = Input(shape=(1,))
vod_input = Input(shape=(1,))
user_embedding = Embedding(len(user_ids), 50)(user_input)
vod_embedding = Embedding(len(vod_ids), 50)(vod_input)
user_vec = Flatten()(user_embedding)
vod_vec = Flatten()(vod_embedding)
concat = Concatenate()([user_vec, vod_vec])
fc1 = Dense(100, activation='relu')(concat)
output = Dense(1)(fc1)
model = Model([user_input, vod_input], output)

In [90]:
# 모델 컴파일
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

In [91]:
# 모델 학습
model.fit([train_data['user'], train_data['vod']], train_data['cd_score'], epochs=5, batch_size=32)

Epoch 1/5
177/177 [==============================] - 2s 5ms/step - loss: 0.0098
Epoch 2/5
177/177 [==============================] - 1s 5ms/step - loss: 0.0068
Epoch 3/5
177/177 [==============================] - 1s 5ms/step - loss: 0.0048
Epoch 4/5
177/177 [==============================] - 1s 5ms/step - loss: 0.0034
Epoch 5/5
177/177 [==============================] - 1s 8ms/step - loss: 0.0024


In [92]:
# 모델 평가
model.evaluate([test_data['user'], test_data['vod']], test_data['cd_score'])

19/19 [==============================] - 0s 3ms/step - loss: 0.0194


0.019418181851506233

In [93]:
# 모델을 사용하여 각 사용자에 대한 아이템의 예측 점수를 계산
def predict_scores(model, user_ids, item_ids):
    predictions = model.predict([user_ids, item_ids])
    return predictions.flatten()

# Precision@k와 Recall@k 계산
def calculate_precision_recall_at_k(model, test_data, k=10):
    precisions = []
    recalls = []

    # 고유한 사용자 ID 목록
    user_ids = test_data['user'].unique()

    for user_id in user_ids:
        # 테스트 데이터에서 해당 사용자의 실제 상호작용 아이템
        actual_items = test_data[test_data['user'] == user_id]['vod'].tolist()

        # 모델을 사용하여 예측 점수 계산
        predicted_scores = predict_scores(model, np.array([user_id] * len(vod_ids)), np.arange(len(vod_ids)))

        # 상위 k개의 예측 아이템의 인덱스를 얻음
        top_k_items = np.argsort(-predicted_scores)[:k]

        # Precision과 Recall 계산
        num_correct_predictions = len(set(top_k_items) & set(actual_items))
        precision = num_correct_predictions / k
        recall = num_correct_predictions / len(actual_items) if len(actual_items) > 0 else 0

        precisions.append(precision)
        recalls.append(recall)

    # 평균 Precision@k와 Recall@k 반환
    return np.mean(precisions), np.mean(recalls)

# Precision@k와 Recall@k 계산
precision_at_k, recall_at_k = calculate_precision_recall_at_k(model, test_data, k=10)

print(f"Precision@10: {precision_at_k}")
print(f"Recall@10: {recall_at_k}")


84/84 [==============================] - 0s 2ms/step
Precision@10: 0.0003448275862068966
Recall@10: 0.0017241379310344827


In [94]:
# 테스트 데이터에서 모든 사용자 ID 추출
all_user_ids = test_data['user'].unique()

# 추천 결과 저장용 데이터프레임 생성
recommendations_df = pd.DataFrame(columns=['user'] + [f'recommended_vod_{i+1}' for i in range(10)])

# 각 사용자에 대해 상위 10개의 아이템 예측
for user_id in all_user_ids:
    # 해당 사용자에 대한 모든 아이템 인덱스 생성
    all_item_indices = np.arange(len(vod_ids))

    # 해당 사용자에 대한 모든 아이템 예측 점수 계산
    predicted_scores = predict_scores(model, np.array([user_id] * len(vod_ids)), all_item_indices)

    # 상위 10개의 아이템 인덱스 추출
    top_10_indices = np.argsort(-predicted_scores)[:10]

    # 각 결과를 데이터프레임에 추가
    recommended_vod_ids = [vod_ids[idx] for idx in top_10_indices]
    row_data = [user_id] + recommended_vod_ids
    recommendations_df = recommendations_df.append(pd.Series(row_data, index=recommendations_df.columns), ignore_index=True)

recommendations_df

22/84 [======>.......................] - ETA: 0s

<ipython-input-94-f7abcbf701d1>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append(pd.Series(row_data, index=recommendations_df.columns), ignore_index=True)


21/84 [======>.......................] - ETA: 0s

<ipython-input-94-f7abcbf701d1>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append(pd.Series(row_data, index=recommendations_df.columns), ignore_index=True)


20/84 [======>.......................] - ETA: 0s

...

19/84 [=====>........................] - ETA: 0s

<ipython-input-94-f7abcbf701d1>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append(pd.Series(row_data, index=recommendations_df.columns), ignore_index=True)


84/84 [==============================] - 0s 3ms/step


<ipython-input-94-f7abcbf701d1>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append(pd.Series(row_data, index=recommendations_df.columns), ignore_index=True)


,user,recommended_vod_1,recommended_vod_2,recommended_vod_3,recommended_vod_4,recommended_vod_5,recommended_vod_6,recommended_vod_7,recommended_vod_8,recommended_vod_9,recommended_vod_10
0,546,360,955,498,1069,2455,450,991,849,1233,816
1,94,450,2455,858,1069,991,2226,955,1233,726,1168
2,512,1069,2455,1233,112,450,1206,955,498,880,360
3,442,2455,498,1233,1206,1069,991,955,849,1168,360
4,253,2455,1233,450,1168,726,1206,2066,858,15,991
...,...,...,...,...,...,...,...,...,...,...,...
285,21,498,1233,450,1069,955,2455,724,849,1206,976
286,242,2455,450,724,498,112,1206,955,1233,830,858
287,764,955,1233,1069,498,849,976,2455,991,954,726
288,669,955,1069,2455,849,757,976,1233,991,498,816


In [96]:
# recommendations_df 데이터프레임의 각 행에 대해
for index, row in recommendations_df.iterrows():
    user_value = row['user']

    # 해당 'user' 값을 가지고 있는 score_filtered 데이터프레임의 행을 찾기
    matching_rows = score_filtered[score_filtered['user'] == user_value]

    # 매칭된 행이 존재하면
    if not matching_rows.empty:
        # 첫 번째 매칭된 행의 인덱스를 가져와서 해당 인덱스의 'user' 값을 'subsr' 값으로 치환
        matching_index = matching_rows.index[0]
        recommendations_df.at[index, 'user'] = score_filtered.at[matching_index, 'subsr']

#
recommendations_df['algorithm'] = 1

# 결과 출력
recommendations_df

,user,recommended_vod_1,recommended_vod_2,recommended_vod_3,recommended_vod_4,recommended_vod_5,recommended_vod_6,recommended_vod_7,recommended_vod_8,recommended_vod_9,recommended_vod_10
0,59879000,360,955,498,1069,2455,450,991,849,1233,816
1,59882000,450,2455,858,1069,991,2226,955,1233,726,1168
2,59892000,1069,2455,1233,112,450,1206,955,498,880,360
3,59897000,2455,498,1233,1206,1069,991,955,849,1168,360
4,59900000,2455,1233,450,1168,726,1206,2066,858,15,991
...,...,...,...,...,...,...,...,...,...,...,...
285,67107000,498,1233,450,1069,955,2455,724,849,1206,976
286,67130000,2455,450,724,498,112,1206,955,1233,830,858
287,67140000,955,1233,1069,498,849,976,2455,991,954,726
288,67149000,955,1069,2455,849,757,976,1233,991,498,816


In [95]:
#recommendations_df.to_csv('recommendations_df.csv', encoding='cp949', index=False)